In [2]:
# Datamart server running on localhost in development mode
datamart_api_url = 'http://localhost:5000'

In [14]:
import os
import json
import pandas as pd
from io import StringIO
from IPython.display import display, HTML
from requests import get,post,put,delete

In [4]:
def upload_data_post(file_path, url):
    file_name = os.path.basename(file_path)
    files = {
        'file': (file_name, open(file_path, mode='rb'), 'application/octet-stream')
    }
    response = post(url, files=files)

    print(json.dumps(response.json(), indent=2))

In [16]:
def upload_data_put(file_path, url):
    file_name = os.path.basename(file_path)
    files = {
        'file': (file_name, open(file_path, mode='rb'), 'application/octet-stream')
    }
    response = put(url, files=files)

    print(json.dumps(response.json(), indent=2))

In [17]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables')
print(json.dumps(response.json(), indent=2))


[
  {
    "name": "INGO",
    "variable_id": "ingo",
    "description": "INGO in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-004",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  },
  {
    "name": "UN",
    "variable_id": "un",
    "description": "UN in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-003",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  }
]


In [22]:
url = f'{datamart_api_url}/datasets/DeleteMe/annotated'
file_path = 'test/test_data/03_worker_incidents_tbd.xlsx'
upload_data_put(file_path, url)

[
  {
    "name": "UN",
    "variable_id": "un",
    "dataset_id": "DeleteMe",
    "description": "UN in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-003",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  },
  {
    "name": "INGO",
    "variable_id": "ingo",
    "dataset_id": "DeleteMe",
    "description": "INGO in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-004",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  }
]


In [20]:
url = f'{datamart_api_url}/datasets/DeleteMe/annotated'
file_path = 'test/test_data/03_worker_incidents_tbd.xlsx'
upload_data_post(file_path, url)

[
  {
    "name": "UN",
    "variable_id": "un",
    "dataset_id": "DeleteMe",
    "description": "UN in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-003",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  },
  {
    "name": "INGO",
    "variable_id": "ingo",
    "dataset_id": "DeleteMe",
    "description": "INGO in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-004",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  }
]


In [11]:
response = get(f'{datamart_api_url}/datasets/DeleteMe/variables/un')
df = pd.read_csv(StringIO(response.text))
display(HTML(df.fillna('').to_html(index=False)))

location_id,dataset_id,variable_id,variable,main_subject,main_subject_id,value,value_unit,time,time_precision,country,admin1,admin2,admin3,coordinate,stated_in,stated_in_id,location,stated in
Q115,DeleteMe,un,UN,Incident ID 103,QQDeleteMe_Incident_ID_103,0.0,count,2000-02-04T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1060,QQDeleteMe_Incident_ID_1060,1.0,count,2010-06-18T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1069,QQDeleteMe_Incident_ID_1069,2.0,count,2010-03-23T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1102,QQDeleteMe_Incident_ID_1102,4.0,count,2011-05-13T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1281,QQDeleteMe_Incident_ID_1281,1.0,count,2011-08-05T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1479,QQDeleteMe_Incident_ID_1479,0.0,count,2012-10-28T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1689,QQDeleteMe_Incident_ID_1689,0.0,count,2013-10-29T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1713,QQDeleteMe_Incident_ID_1713,1.0,count,2013-07-22T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1923,QQDeleteMe_Incident_ID_1923,0.0,count,2014-08-09T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,
Q115,DeleteMe,un,UN,Incident ID 1924,QQDeleteMe_Incident_ID_1924,0.0,count,2014-08-07T00:00:00Z,,Ethiopia,,,,"POINT(40.0, 9.0)",,,Ethiopia,


In [37]:
response = delete(f'{datamart_api_url}/datasets/DeleteMe/variables')
print(response.text)
# print(json.dumps(response.json(), indent=2))

{"message": "The method is not allowed for the requested URL."}



In [48]:
response = delete(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables/ingo')
print(response.text)
# print(json.dumps(response.json(), indent=2))

{"Message": "Variable: ingo in the dataset: DeleteMe has been deleted."}



In [12]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables')
print(len(response.json()))
print(json.dumps(response.json(), indent=2)) 

2
[
  {
    "name": "INGO",
    "variable_id": "ingo",
    "description": "INGO in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-004",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  },
  {
    "name": "UN",
    "variable_id": "un",
    "description": "UN in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-003",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  }
]
